In [4]:
from rtree import index
import pickle

scaler_path = "./bin/scaler.dat"
pca_path = "./bin/pca.dat"
ncomponents_path = "./bin/ncomponents.dat"

scaler = pickle.load(open(scaler_path, "rb"))
pca = pickle.load(open(pca_path, "rb"))
ncomponents = pickle.load(open(ncomponents_path, "rb"))

path = "./bin/rtree_index"
p = index.Property()
p.dimension = ncomponents #D

idx = index.Index(path, properties=p)

In [19]:
import pandas as pd
import numpy as np
def generate_df(x):
    return pd.DataFrame(data=x, columns = [str(i) for i in range(1, 129)])

def generate_point(v):
    return tuple(np.concatenate([v, v]))

In [14]:
import face_recognition
image_path = "test/Roger_Moore_0004.jpg"
picture = face_recognition.load_image_file(image_path)    
all_face_encodings = face_recognition.face_encodings(picture)
x = generate_df(all_face_encodings)

In [20]:
x_scaled = scaler.transform(x)
x_pca = pca.transform(x_scaled)
point = generate_point(x_pca[0])

In [21]:
lres = list(idx.nearest(coordinates=point, num_results=5, objects="raw"))
print("Resultado Knn: ", lres)

Resultado Knn:  ['Roger_Moore/Roger_Moore_0004.jpg', 'Roger_Moore/Roger_Moore_0001.jpg', 'Roger_Moore/Roger_Moore_0003.jpg', 'Roger_Moore/Roger_Moore_0002.jpg', 'Enrique_Bolanos/Enrique_Bolanos_0003.jpg']
